In [20]:
import os
import sys
import json
import time
import datetime
from importlib import reload

import requests

import pandas as pd

import external as ext

In [12]:
# Put the apikey provided during the subscription to the pubg developer portal
apikey = ""

In [13]:
#Define the headers for the request
headers = {
    "Authorization": f'Bearer {apikey}',
    "Accept": "application/vnd.api+json"
}

## Collect the matchid from the sample endpoint

In [14]:
# Define the area
area = "pc-na"

In [17]:
# Setup the request
url_collectmatchid = f"https://api.pubg.com/shards/{area}/samples"
response = requests.get(url_collectmatchid, headers=headers)

#Format the data (dirty way)
try:
    matches = [match["id"] for match in response.json()["data"]['relationships']["matches"]["data"]]
except:
    print(response.text)
    matches = []
    
# Quick check
nbr_matches = len(matches)
print("Nbr of matches",nbr_matches)
print("Sample",matches[:5])

Nbr of matches 1166
Sample ['b551a96e-c8c3-4f4a-a4ee-520f5a8305a8', 'bcc7a55a-59c9-4e7c-8ad5-84bab20b0801', '929d0e59-92c1-4e7e-b5ae-3de2b78b7258', '024c8277-04ba-4e2d-a6a1-e5df18e97d78', '049c3bd1-67a7-420b-8c22-9edf7dee9707']


## Collect the details on the match

In [18]:
#Choose one match ofr the test
matchid = matches[1]

In [24]:
reload(ext)
#Collect the details on the match
df_match,url_eventsmatch,typ = ext.get_matchdetails(area,matchid,headers)
print("Number of players:",len(df_match))
print("Url of the match:",url_eventsmatch)
print("Type of match:",typ)

Number of players: 17
Url of the match: https://telemetry-cdn.playbattlegrounds.com/bluehole-pubg/pc-na/2019/01/28/00/01/d6dfdb2a-228f-11e9-8cba-0a5864607e24-telemetry.json
Type of match: solo


## Collect the events associated to the match

In [25]:
df_events = ext.get_eventsdetails(url_eventsmatch)

In [26]:
df_events.head()

,tstp,type_event,details
1,2019-01-27 23:27:59.788,LogPlayerLogin,{'accountId': 'account.7bbd1f5ffcbb45d49135ea8...
2,2019-01-27 23:27:59.819,LogPlayerCreate,"{'character': {'name': 'Kr0w', 'teamId': 10000..."
3,2019-01-27 23:28:00.794,LogPlayerLogin,{'accountId': 'account.a907a52a493c44c49e1122f...
4,2019-01-27 23:28:00.803,LogPlayerCreate,"{'character': {'name': 'carolinanavy', 'teamId..."
5,2019-01-27 23:28:02.030,LogPlayerLogin,{'accountId': 'account.8b2ebc9844424007af0798f...


## Get general details on the match

In [28]:
ext.get_detailsmatch(df_events)

{'start_date': '2019-01-27 23:29:47',
 'end_date': '2019-01-27 23:59:56',
 'nbr_players': 17,
 'duration': 1808,
 'mapname': 'Range_Main',
 'weatherid': 'Weather_Range_Sunset',
 'cameraviewbehaviour': 'FpsAndTps',
 'teamsize': 1,
 'iscustomgame': True,
 'iseventmode': False,
 'bluezonecustomoptions': '[]',
 'matchid': 'match.bro.training.training.steam.solo.na.2019.01.27.23.bcc7a55a-59c9-4e7c-8ad5-84bab20b0801',
 'pingquality': 'low',
 'seasonstate': 'progress'}

## Make an event segmentation

In [29]:
#Select the events that are not match related (just one event in all the match)
df_events_toclean = df_events[df_events['type_event'].str.contains("Match") == False]

In [31]:
#For each type of event
for type_event in df_events_toclean['type_event'].unique()[:2]:
    print(type_event)
    df_events_toclean_selected = df_events_toclean[df_events_toclean ["type_event"] == type_event]

    clean_events = []
    for i,row in df_events_toclean_selected.iterrows():
        raw_details = row["details"]
        #print(raw_details)
        raw_details["tstp"] = row["tstp"]
        raw_details = ext.get_cleandict(raw_details)
        clean_events.append(raw_details)

    df_events_clean = pd.DataFrame(clean_events)
    df_events_clean.sort_values(['tstp'], ascending= True, inplace = True)
    df_events_clean.reset_index(drop = True, inplace = True)
    
    print(df_events_clean.head())

LogPlayerLogin
                                  accountId  common_isgame  \
0  account.7bbd1f5ffcbb45d49135ea8a908fe0fe              0   
1  account.a907a52a493c44c49e1122f71b5816e0              0   
2  account.8b2ebc9844424007af0798f37fe64bd8              0   
3  account.541f83b2dc2241a5a860268582f22e4f              0   
4  account.d7707b5d692a4cb382256c9b11118b78              0   

                     tstp  
0 2019-01-27 23:27:59.788  
1 2019-01-27 23:28:00.794  
2 2019-01-27 23:28:02.030  
3 2019-01-27 23:28:07.794  
4 2019-01-27 23:28:11.691  
LogPlayerCreate
                        character_accountid  character_health  \
0  account.7bbd1f5ffcbb45d49135ea8a908fe0fe               100   
1  account.a907a52a493c44c49e1122f71b5816e0               100   
2  account.8b2ebc9844424007af0798f37fe64bd8               100   
3  account.541f83b2dc2241a5a860268582f22e4f               100   
4  account.d7707b5d692a4cb382256c9b11118b78               100   

   character_isinbluezone  character_

## Process to send data in S3

In [32]:
#Define the client to send the data (use the credntials that are setup on your machine but can be defined by the access id)
import boto3
s3_client = boto3.client('s3')
transfer = boto3.s3.transfer.S3Transfer(s3_client)

In [ ]:
#Command to send data in s3
file_local = "name of the file in your machine"
file_s3 = "name of the file in s3"#Can be useful to define a path to create a simple arborescence
s3bucket = "name of the s3 bucket"
transfer.upload_file(file_local, s3bucket, file_s3)